In [35]:
import numpy as np

In [36]:
import pandas as pd

In [37]:
input_path = "../../input"

In [38]:
train_df = pd.read_csv(input_path + "/train/train.csv")

In [39]:
test_df = pd.read_csv(input_path + "/test/test.csv")

In [40]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [41]:
# drop text features and hash id`s
drop_list = ['Name', 'RescuerID', 'Description']

In [42]:
train = train_df.drop(drop_list, axis=1)

In [43]:
test = test_df.drop(drop_list, axis=1)

In [44]:
x_train = train.drop(['PetID', 'AdoptionSpeed'], axis=1)

In [45]:
y_train = train['AdoptionSpeed']

In [46]:
from catboost import CatBoostClassifier, Pool

In [47]:
from sklearn.model_selection import StratifiedKFold, ParameterGrid, train_test_split

In [48]:
from sklearn.metrics import cohen_kappa_score

In [49]:
from tqdm import tqdm

In [50]:
# for tuning parameters show simple_catboost_tuning_param

In [51]:
x_test = test.drop(['PetID'], axis=1)

In [52]:
cat_features = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,17]

In [53]:
clf = CatBoostClassifier(iterations=500,
                            loss_function = 'MultiClass',
                            depth=4,
                            leaf_estimation_iterations = 10,
                            use_best_model=True,
                            classes_count=5,
                            cat_features=cat_features
    )

In [54]:
X_train, X_valid, y_train, y_valid = train_test_split(x_train,
                                                        y_train, 
                                                        train_size=0.8,
                                                        stratify=y_train
    )

In [55]:
clf.fit(X_train, 
            y_train,
            logging_level='Silent',
            eval_set=(X_valid, y_valid)
    )

In [56]:
test_pet_id = test['PetID']

In [57]:
pred_test = clf.predict(x_test)

In [58]:
pred = list(map(lambda x: int(x[0]), pred_test))

In [59]:
submit_df = pd.DataFrame(data={'PetID': test_pet_id, 'AdoptionSpeed': pred})

In [60]:
submit_df.head()

,PetID,AdoptionSpeed
0,378fcc4fc,2
1,73c10e136,4
2,72000c4c5,4
3,e147a4b9f,4
4,43fbba852,4


In [29]:
submit_df.to_csv('submission.csv', index=False)